# Introduction to Julia, 
## SIAM Student Chapter KU Leuven

This is a small tutorial to get to know some of the interesting functions of Julia, assuming the reader has prior general knowledge in programming languages (C++, Python, Matlab). 

This notebook is a collection of small code snippets illustrating different aspects of Julia.

Most examples (~98%) where taken from [the notebooks of Daan Huybrechs](https://github.com/daanhb/Julia-tutorial).

### Basic syntax
#### 1. Variables

You don't declare any types, but Julia will always deduce them (and if it can't, the type of your variable will be Any). Read all about variables [in the documentation](https://docs.julialang.org/en/stable/manual/variables/ "Documentation").

In [2]:
a = 2

2

In [3]:
typeof(a)

Int64

In [4]:
a = 8.0
typeof(a)

Float64

In [6]:
b = [2, 5.0, "hello"]

3-element Array{Any,1}:
 2       
 5.0     
  "hello"

##### Variables are always references!

This is important to note from the start: variables in Julia are just names associated with a value. This means we always have **reference semantics**! This is (very) different from Matlab.

For example:

In [7]:
a = [1, 2, 3, 4]

4-element Array{Int64,1}:
 1
 2
 3
 4

In [8]:
b = a
b[2] = 5
b

4-element Array{Int64,1}:
 1
 5
 3
 4

In [9]:
a

4-element Array{Int64,1}:
 1
 5
 3
 4

You can avoid reference semantics with an explicit copy:

In [10]:
c = copy(b)
c[2] = 7
c

4-element Array{Int64,1}:
 1
 7
 3
 4

In [11]:
b

4-element Array{Int64,1}:
 1
 5
 3
 4

Since arrays are always passed by references, it becomes easy to make functions that modify their arguments - you can make **in-place algorithms**. Note for future reference the Julia convention of naming functions that modify one of their arguments in a particular way, namely ending in an exclamation point. For example, [see](http://docs.julialang.org/en/release-0.4/stdlib/collections/?highlight=push!#Base.push!) the `push!` function that changes a vector by adding an element to it.

More importantly, reference semantics lets you avoid making unnecessary copies of data all over the place. Matlab avoids the copying using a copy-on-write mechanism. This is a valid choice to make, but it is still less efficient than just treating everything as a reference.

In [12]:
a = 4
b = a
b = 5

5

Modifying b did not modify a. Why? Because we did not actually modify b at all! The line b = 5 simply made b refer to a different integer, unrelated to what it was pointing at before. It is a new assignment, no different from the line a = 4 above.

In any case, you can not change the value of an integer, because it is immutable. Arrays are mutable, in the sense that you can alter the third entry of an array. This does not make integers special in any way. Julia has mutable and immutable types, we will see that later.


#### 2. More on vectors
Vector can easily be generated using square brackets (see above), or using **list comprehensions**.

In [13]:
a = [factorial(i) for i=1:10]

10-element Array{Int64,1}:
       1
       2
       6
      24
     120
     720
    5040
   40320
  362880
 3628800

In [14]:
A = [1/(i+j) for i=1:4,j=1:4]

4×4 Array{Float64,2}:
 0.5       0.333333  0.25      0.2     
 0.333333  0.25      0.2       0.166667
 0.25      0.2       0.166667  0.142857
 0.2       0.166667  0.142857  0.125   

#### 3. Linear algebra
Most linear algebra operations you may know from Matlab also exist in Julia. 

In [17]:
Q,R = qr(A)

4×4 Array{Float64,2}:
 -0.68089  -0.489555   -0.384651    -0.317628  
  0.0      -0.0415261  -0.0522176   -0.0539782 
  0.0       0.0        -0.00177233  -0.00310257
  0.0       0.0         0.0          4.71355e-5

In [21]:
eig(A)

([2.13066e-5, 0.00182126, 0.0622678, 0.977556], [-0.0568608 -0.272282 0.662773 -0.695242; 0.407652 0.738785 -0.188579 -0.502448; -0.791397 0.0491555 -0.463099 -0.395998; 0.451971 -0.614526 -0.557413 -0.327674])

### Control flow
The famous `for` and `if` statements work as expected.

In [24]:
cars = ["Porsche", "Audi", "Tesla"]
for brand in cars
   println(brand) 
end

Porsche
Audi
Tesla


The following type of Matlab style ranges (for example 1:5) are possible.

In [27]:
for i in 1:5
   println("Hi from iteration $i") 
end

Hi from iteration 1
Hi from iteration 2
Hi from iteration 3
Hi from iteration 4
Hi from iteration 5


In [28]:
if 2<3
    println("Statement is true.")
else
    println("Trump is a lie.")
end

Statement is true.


### Functions
Functions are a bit like in Matlab, except that you don't specify output variables. The last expression that is evaluated yields the return value (like it is in, say, Maple).

In [30]:
function fibonacci(n)
    if (n == 1) || (n==0)
        1
    else
        fibonacci(n-1) + fibonacci(n-2)
    end
end

fibonacci (generic function with 1 method)

In [31]:
fibonacci(5)

8

You can also explicitly write `return` if you like

In [29]:
function my_maximum(x,y)
    if x > y
        return x
    else
        return y
    end
end
my_maximum(2,3)

3

Finally, you can create anonymous functions.


In [34]:
x -> cos(x)

(::#3) (generic function with 1 method)

You can pass around functions as arguments, including operators (which are really just functions).

In [35]:
composite(f, g, x) = f(g(x))

composite (generic function with 1 method)

In [36]:
composite(-, -, 1)

1

### Type system

We have already encounted a few types, including the numeric types Int64, Float64. Writing Julia revolves a lot around types. You will be making lots of them, and using them all the time. You can be generous with new types, they are not scarce. Sometimes code you write creates types implicitly just to get things done - in that case the type is essentially a use-once-throw-away commodity.

#### 1. Composite type
You can compare a Julia type with a struct in C: it is an object that collects data. Member data are called fields. The data can be named and you define a type by listing the names of its fields.

In [38]:
type MyType
    a
end

In [39]:
v = MyType(5)    # We instantiate an object of type MyType. There is a default constructor.

MyType(5)

In [40]:
v.a

5

Use untyped fields when you care about flexibility and simplicity more than performance. They do have unavoidable runtime overhead, so don't use untyped fields in a time-critical path of your code. In that case, do the following:


In [42]:
type MyType2
    a :: Float64
end

In [43]:
v = MyType2(2)      # Note the integer I've supplied is automatically converted to a float

MyType2(2.0)

In [44]:
v.a = "I'm an evil string."

LoadError: [91mMethodError: Cannot `convert` an object of type String to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.[39m

There is a difference in runtime. Let's read the field's value of the declared type many times.

In [47]:
function add_field_values_many_times(m)
    z = 0.0
    for i = 1:10000
        z = z + m.a
    end
end

add_field_values_many_times (generic function with 1 method)

In [48]:
add_field_values_many_times(MyType(10.0))
@time add_field_values_many_times(MyType(10.0))

  0.000199 seconds (10.09 k allocations: 162.717 KiB)


In [49]:
add_field_values_many_times(MyType2(10.0))
@time add_field_values_many_times(MyType2(10.0))

  0.000009 seconds (5 allocations: 176 bytes)


There is much more too say about it (immutable types,etc ), be sure to check the documentation

#### 2. Parametric types

In [51]:
immutable Point{T}
    x :: T
    y :: T
    z :: T
end

In [52]:
p = Point(0.1, 0.2, 0.4)

Point{Float64}(0.1, 0.2, 0.4)

In [53]:
p.x

0.1

Why would I define a point this way? I want to specify the types of x, y and z, so that Julia knows them and produces optimized code. But I don't want to specify that they are Float64. What if my user wants to use BigFloat's? Or integers? Or something else entirely I do not know about, some user-defined numeric type? In this case, parameters are the answer.

Type parameters in Julia look a lot like C++ template parameters. A major difference is that C++ templates are mostly syntactic sugar at compile-time. You could achieve what they do, if you have the patience, by using copy-paste of text over and over again. In Julia, parametric types and methods are a major part of the language in all stages of execution.

I am skipping over many things here, like default and user-supplied constructors. There are inner and outer constructors. They can be painful at times. Please read the manual on [constructors](http://docs.julialang.org/en/stable/manual/constructors/), especially [parametric constructors](http://docs.julialang.org/en/stable/manual/constructors/#parametric-constructors), and don't complain to me.


##### 3. Abstract types
Sure enough, types can inherit from other types. You can create an abstract type and then inherit from it.

In [55]:
abstract type AbstractPoint 
end

In [56]:
immutable Point2d <: AbstractPoint
    x :: Float64
    y :: Float64
end

In [57]:
immutable Point3d <: AbstractPoint
    x :: Float64
    y :: Float64
    z :: Float64
end

In [58]:
p2 = Point2d(0.1, 2.0)
typeof(p2)

Point2d

In [62]:
supertype(Point3d)


AbstractPoint

Any type that is not abstract is a concrete type. You can not inherit from concrete types. For example, we could not create a subset of Point2d's, say a subset that has unit norm, as follows:


In [63]:
type Point2d_with_unit_norm <: Point2d
end

LoadError: [91minvalid subtyping in definition of Point2d_with_unit_norm[39m

### Multiple dispatch
But what are types? All variables have a type. It is compiler metadata that describes what your variable stands for. Julia types compare a little bit to classes in an object-oriented language, but one should not stretch the comparison too much. Types do support inheritance. However, you are well advised to resist that initial urge to create a fancy hierarchical tree of types like you would in OOP: there is no need for that. The best reason to use types in Julia (and some would say the only reason) is to use multiple dispatch, and that is the topic of the next chapter.

Without multiple dispatch it is hard to convey the richness of the type system. Still, we have a quite a few things to discover already. 

#### Methods are algorithms are methods: Julia selects the best available algorithm for the problem at hand
Function overloading is possible in many typed languagues, but in Julia it is really pervasive. You add methods to existing functions all the time. Popular ones are the basic operators, like +:


In [65]:
methods(+)

# 180 methods for generic function "+":
+(x::Bool, z::Complex{Bool}) in Base at complex.jl:232
+(x::Bool, y::Bool) in Base at bool.jl:89
+(x::Bool) in Base at bool.jl:86
+(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:96
+(x::Bool, z::Complex) in Base at complex.jl:239
+(a::Float16, b::Float16) in Base at float.jl:372
+(x::Float32, y::Float32) in Base at float.jl:374
+(x::Float64, y::Float64) in Base at float.jl:375
+(z::Complex{Bool}, x::Bool) in Base at complex.jl:233
+(z::Complex{Bool}, x::Real) in Base at complex.jl:247
+(x::Char, y::Integer) in Base at char.jl:40
+(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:312
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:334
+(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:327
+(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:321
+(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:289
+(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:346
+(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:362
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:460
+(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:453
+(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:447
+(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:308
+(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:277
+(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:284
+(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:292
+(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:300
+(B::BitArray{2}, J::UniformScaling) in Base.LinAlg at linalg/uniformscaling.jl:59
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem, b::Base.Pkg.Resolve.VersionWeights.VWPreBuildItem) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:87
+(a::Base.Pkg.Resolve.VersionWeights.VWPreBuild, b::Base.Pkg.Resolve.VersionWeights.VWPreBuild) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:135
+(a::Base.Pkg.Resolve.VersionWeights.VersionWeight, b::Base.Pkg.Resolve.VersionWeights.VersionWeight) in Base.Pkg.Resolve.VersionWeights at pkg/resolve/versionweight.jl:197
+(a::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue, b::Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue) in Base.Pkg.Resolve.MaxSum.FieldValues at pkg/resolve/fieldvalue.jl:44
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.CompoundPeriod) in Base.Dates at dates/periods.jl:349
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.Period) in Base.Dates at dates/periods.jl:347
+(x::Base.Dates.CompoundPeriod, y::Base.Dates.TimeType) in Base.Dates at dates/periods.jl:387
+(x::Date, y::Base.Dates.Day) in Base.Dates at dates/arithmetic.jl:77
+(x::Date, y::Base.Dates.Week) in Base.Dates at dates/arithmetic.jl:75
+(dt::Date, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:58
+(dt::Date, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:32
+(dt::Date, t::Base.Dates.Time) in Base.Dates at dates/arithmetic.jl:20
+(t::Base.Dates.Time, dt::Date) in Base.Dates at dates/arithmetic.jl:24
+(x::Base.Dates.Time, y::Base.Dates.TimePeriod) in Base.Dates at dates/arithmetic.jl:81
+(dt::DateTime, z::Base.Dates.Month) in Base.Dates at dates/arithmetic.jl:52
+(dt::DateTime, y::Base.Dates.Year) in Base.Dates at dates/arithmetic.jl:28
+(x::DateTime, y::Base.Dates.Period) in Base.Dates at dates/arithmetic.jl:79
+(y::AbstractFloat, x::Bool) in Base at bool.jl:98
+(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:32
+(x::Integer, y::Ptr) in Base at pointer.jl:128
+(z::Complex, w::Complex) in Base at complex.jl:221
+(z::Complex, x::Bool) in Base at complex.jl:240
+(x::Real, z::Complex{Bool}) in Base at complex.jl:246
+(x::Real, z::Complex) in Base at complex.jl:258
+(z::Complex, x::Real) in Base at complex.jl:259
+(x::Rational, y::Rational) in Base at rational.jl:245

A good design of Julia code makes sure that different problems map to different types, or that different properties of a problem match to different types. You write the algorithms in terms of these types, and Julia makes sure that the right algorithm is called at the right time. As you see, this is not entirely for free. It is not really the case that Julia selects the right algorithm. Julia selects the method that best matches the type signature. It is up to you, the programmer, to make sure that this corresponds to the right algorithm.

This does mean that multiple dispatch affects the design of your code. You design data structures with dispatch in mind, and you write algorithms that apply to types. This is not difficult to achieve once you're used to it, but it is quite pervasive and it is so pretty much from the start.

### More fun
Macro's + parallel code